# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store quantum circuits, experiments, build experiment, and excute them on local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy on local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install qurrium
```


In [1]:
from IPython.display import display
from qurry import __version__

print(f"| Current Version: {__version__}")

| Current Version: 0.9.1.dev1


## 1.1 Setup Environment and Creat experiment executor


In [2]:
from qurry import EntropyMeasure
from qiskit import QuantumCircuit

- Import simulator


In [3]:
from qiskit.providers.basic_provider import BasicProvider

basic_provider = BasicProvider()
backend_sim = basic_provider.get_backend("basic_simulator")

- Next, we will initialize our `EntropyMeasure` instance. There are two methods implemented based on two theories. The defaul method is `randomized` based on Randomized Measure. The other method is `hadamard` based on Hadamard Test. 

- In the following tutorial, we will use `randomized` as our measurement method.


In [4]:
experiment_hadamard = EntropyMeasure(method="hadamard")
print('| The first method is which is `Hadamard Test`".')
experiment_randomized = EntropyMeasure()
print("| The default method is which is `Randomized Measure` we will use.")

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuits

- In `Qurry`, there are built-in quantum circuits that construct certain wave functions, or equivalently, quantum states. They can be imported from `qurry.recipe`.
- The user can define customized circuits following Qiskit workflow.



In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)
sample03 = QuantumCircuit(8)
sample03.x(range(0, 8, 2))
print("| Custom circuit:")
print(sample03)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

- After we prepare three quantum circuits, we append them to the `EntropyMeasure` object by the `.add()` method.
- The attribute `.waves` of `Qurry` object is a customized `dict`. we can check the stored circuits by printing the dictionary.

In [7]:
experiment_randomized.add(
    wave=sample01,  # The circuit object
    key="TrivialParamagnet",  # The name of the circuit
    replace=False,  # True for if you want to replace the circuit with the same name.
)  # Default is False, so if you add the same name when False,
# it will be added with serial number.
experiment_randomized.add(sample02, "GHZ")
experiment_randomized.add(sample03)
print(experiment_randomized.waves)

{'TrivialParamagnet': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x75f0be15c770>, 'GHZ': <qurry.recipe.simple.cat.GHZ object at 0x75f0c7f8f8c0>, 2: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x75f0c0b73800>}


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.


## 1.3 Excute Circuits


- We can use `.measure()` method to excute the specific circuit in the `EntropyMeasure` object. The `.measure()` method  returns a hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- `EntropyMeasure.measure()` requires the following input parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurements
  - shots: the number of shots for each measurement. The default is 1024.


In [8]:
exp1 = experiment_randomized.measure(
    wave="TrivialParamagnet", times=100, shots=1024, backend=backend_sim
)
exp1

'4939d7d1-c2a5-4bb9-9de8-3194a3ce199b'

- You can specify the backend by add a parameter `backend` in `measure` method, if you do not specify, it will use simulator as the default backend. Moreover, qiskit has multiple simulators, they are `QasmSimulatorPy` from `qiskit`, `AerSimulator` from `qiskit-aer`, and `AerSimulator` from `qiskit-aer-gpu`. Qurry will check the availability of these simulators and use the first available one as the default backend in following order:
  1. `AerSimulator` from `qiskit-aer-gpu`
  2. `AerSimulator` from `qiskit-aer`
  3. `QasmSimulatorPy` from `qiskit`


In [9]:
exp2 = experiment_randomized.measure(
    wave="TrivialParamagnet",
    times=100,
    shots=1024,
)
exp2

'ac02024c-f2d4-4ff0-a1dd-7867e6b49f37'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.


In [10]:
print(experiment_randomized.exps)

{'4939d7d1-c2a5-4bb9-9de8-3194a3ce199b': <EntropyMeasureRandomizedExperiment(exp_id=4939d7d1-c2a5-4bb9-9de8-3194a3ce199b, EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomized', times=100, measure=(0, 8), unitary_loc=(0, 8), random_unitary_seeds=None, workers_num=16), Commonparams(exp_id='4939d7d1-c2a5-4bb9-9de8-3194a3ce199b', target_keys=['TrivialParamagnet'], shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x75f0be133410>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2024-10-07 03:21:12', 'run.001': '2024-10-07 03:21:12'})), unused_args_num=0, analysis_num=0)>, 'ac02024c-f2d4-4ff0-a1dd-7867e6b49f37': <EntropyMeasureRandomizedExperiment(exp_id=ac02024c-f2d4-4ff0-a1dd-7867e6b49f37, EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomize

- The following is the other informatiom you can access.


In [11]:
experiment_randomized.waves

WaveContainer({
  'TrivialParamagnet': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x75f0be15c770>,
  'GHZ': <qurry.recipe.simple.cat.GHZ object at 0x75f0c7f8f8c0>,
  2: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x75f0c0b73800>})

In [12]:
experiment_randomized.exps[exp1]

<EntropyMeasureRandomizedExperiment(exp_id=4939d7d1-c2a5-4bb9-9de8-3194a3ce199b, 
  EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomized', times=100, measure=(0, 8), unitary_loc=(0, 8), random_unitary_seeds=None, workers_num=16),
  Commonparams(exp_id='4939d7d1-c2a5-4bb9-9de8-3194a3ce199b', target_keys=['TrivialParamagnet'], shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x75f0be133410>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2024-10-07 03:21:12', 'run.001': '2024-10-07 03:21:12'})),
  unused_args_num=0,
  analysis_num=0))>

In [13]:
print(experiment_randomized.exps[exp1].statesheet())

 # EntropyMeasureRandomizedExperiment with exp_id=4939d7d1-c2a5-4bb9-9de8-3194a3ce199b
 - arguments
   - exp_name --------------------- experiment.N_U_100.qurrent_randomized
   - times ------------------------ 100
   - measure ---------------------- (0, 8)
   - unitary_loc ------------------ (0, 8)
   - random_unitary_seeds --------- None
   - workers_num ------------------ 16
 - commonparams
   - exp_id ----------------------- 4939d7d1-c2a5-4bb9-9de8-3194a3ce199b   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - target_keys ------------------ ['TrivialParamagnet']
   - shots ------------------------ 1024
   - backend ---------------------- <qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x75f0be133410>
   - run_args --------------------- {}
   - transpile_args --------------- {}
   - tags ------------------------- ()
   - default_analysis ------------- []
   - save_location ---------------- .
   - filename ---------------

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.


---

## 1.4 Post-Processing

- To obtain the Renyi Entropy of the whole system and the subsystems. We use `exps[].analyze()` method to calculate and store a `Analysis` object in `Experiment`.
- The `analyze` method requires the following input parameters:
  - degree: The size or selected qubits of the subsystem. Please refer to the following chart for the selection.


### Bit select hint

A bit string mapping to the index of qubit is `'76543210'` for 8 qubits example here.

| the input | bitStringRange | String Slice | Actual Qubit Included |
| --------- | -------------- | ------------ | --------------------- |
| `2`       | `(6, 8)`       | `'10'`       | 01                    |
| `4`       | `(4, 8)`       | `'3210'`     | 0123                  |
| `6`       | `(2, 8)`       | `'543210'`   | 012345                |
| `(2, 4)`  | `(2, 4)`       | `'54'`       | 45                    |
| `(4, 6)`  | `(4, 6)`       | `'32'`       | 23                    |
| `(2, 6)`  | `(2, 6)`       | `'5432'`     | 2345                  |
| `(-2, 2)` | `(-2, 2)`      | `'1076'`     | 6701                  |
| `(-4, 2)` | `(-4, 2)`      | `'321076'`   | 670123                |
| `(-2, 4)` | `(-2, 4)`      | `'107654'`   | 456701                |
| `3`       | `(5, 8)`       | `'210'`      | 012                   |
| `(5, 7)`  | `(5, 7)`       | `'21'`       | 12                    |
| `(4, 7)`  | `(4, 7)`       | `'321'`      | 123                   |


In [14]:
exp1_report1 = experiment_randomized.exps[exp1].analyze(degree=(3, 4))
exp1_report2 = experiment_randomized.exps[exp1].analyze(degree=1)

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.


In [15]:
exp1_report2.export()

({'purity': np.float64(1.0110738563537598),
  'entropy': np.float64(-0.015888386265190953),
  'puritySD': np.float64(0.3859555994472044),
  'entropySD': np.float64(0.5507176610557984),
  'bitStringRange': [7, 8],
  'allSystemSource': "AnalysisHeader(serial=0, datetime='2024-10-07 03:21:15', summoner=None, log={})",
  'purityAllSys': np.float64(1.1983592414855957),
  'entropyAllSys': np.float64(-0.26106046089646756),
  'puritySDAllSys': np.float64(1.28021687418337),
  'entropySDAllSys': np.float64(1.5412427857251338),
  'bitsStringRangeAllSys': [3, 4],
  'errorRate': np.float64(-0.09505119554474291),
  'mitigatedPurity': np.float64(0.9262013471699836),
  'mitigatedEntropy': np.float64(0.11060223942852317),
  'num_qubits': 8,
  'measure': ['measure range:', [0, 8]],
  'measureActually': [0, 8],
  'measureActuallyAllSys': [0, 8],
  'countsNum': 100,
  'takingTime': 0.000532519,
  'takingTimeAllSys': 0,
  'counts_used': None,
  'input': {'degree': 1, 'shots': 1024, 'unitary_loc': [0, 8]},


In [16]:
print(exp1_report1)
print(exp1_report1.header)

qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(3, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=0.8832622337341308, entropy=0.17908626840479677, and others), 0 unused arguments
AnalysisHeader(serial=0, datetime='2024-10-07 03:21:15', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.


In [17]:
print(experiment_randomized.exps[exp1].reports)
print(experiment_randomized.exps[exp1].reports[0])
print(experiment_randomized.exps[exp1].reports[0].header)

AnalysisContainer(length=2, {0{...}, 1{...}})
qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(3, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=0.8832622337341308, entropy=0.17908626840479677, and others), 0 unused arguments
AnalysisHeader(serial=0, datetime='2024-10-07 03:21:15', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.


In [18]:
main, side_prodct = experiment_randomized.exps[exp1].reports[0].export()

- And there is it


In [19]:
main

{'purity': np.float64(0.8832622337341308),
 'entropy': np.float64(0.17908626840479677),
 'puritySD': np.float64(0.34193682311747414),
 'entropySD': np.float64(0.5585097382952265),
 'bitStringRange': [3, 4],
 'allSystemSource': 'independent',
 'purityAllSys': np.float64(1.1983592414855957),
 'entropyAllSys': np.float64(-0.26106046089646756),
 'puritySDAllSys': np.float64(1.28021687418337),
 'entropySDAllSys': np.float64(1.5412427857251338),
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': np.float64(-0.09505119554474291),
 'mitigatedPurity': np.float64(0.8196150190546956),
 'mitigatedEntropy': np.float64(0.2869816735967681),
 'num_qubits': 8,
 'measure': ['measure range:', [0, 8]],
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.000706787,
 'takingTimeAllSys': 0.006651443,
 'counts_used': None,
 'input': {'degree': [3, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2024-10-07 03:21:15',
  'summoner': No

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.


In [20]:
side_prodct

{'purityCells': {54: 0.5394191741943359,
  94: 1.1537094116210938,
  43: 0.504486083984375,
  71: 0.7131404876708984,
  79: 0.5280380249023438,
  56: 1.2049617767333984,
  82: 0.8106441497802734,
  97: 0.7131404876708984,
  8: 1.130706787109375,
  32: 0.5148830413818359,
  25: 0.653900146484375,
  26: 0.533050537109375,
  44: 0.9518184661865234,
  10: 1.392782211303711,
  6: 0.5078334808349609,
  83: 0.6937255859375,
  91: 0.5443115234375,
  95: 1.4063949584960938,
  12: 1.7856063842773438,
  13: 0.6752376556396484,
  81: 1.1383285522460938,
  46: 1.067770004272461,
  87: 0.511077880859375,
  68: 0.5642929077148438,
  73: 1.2833480834960938,
  51: 0.5908432006835938,
  30: 0.82958984375,
  86: 0.6374721527099609,
  76: 0.5018539428710938,
  57: 0.6287460327148438,
  4: 0.685394287109375,
  5: 0.5549545288085938,
  67: 0.8214015960693359,
  80: 1.8570919036865234,
  53: 0.8026962280273438,
  70: 0.738128662109375,
  20: 1.3132686614990234,
  1: 1.0896053314208984,
  49: 0.50252342224121

In [21]:
experiment_randomized.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial ----------------------- 0
   - datetime --------------------- 2024-10-07 03:21:15
   - summoner --------------------- None
   - log -------------------------- {}
 - input
   - degree ----------------------- (3, 4)
   - shots ------------------------ 1024
   - unitary_loc ------------------ (0, 8)
 - outfields -------------------- 0 ........  # Number of unused arguments.
 - content
   - purity ----------------------- 0.8832622337341308
   - entropy ---------------------- 0.17908626840479677
   - puritySD --------------------- 0.34193682311747414
   - entropySD -------------------- 0.5585097382952265
   - purityCells ------------------ {54: 0.5394191741943359, 94: 1.1537094116210938, 43: 0.504486083984375, 71: 0.7131404876708984, 79: 0.5280380249023438, 56: 1.2049617767333984, 82: 0.8106441497802734, 97: 0.7131404876708984, 8: 1.130706787109375, 32: 0.5148830413818359, 25: 0.653900146484375, 26: 0.533050537109375, 44: 0.9

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.


In [22]:
from qurry.process import AVAIBILITY_STATESHEET

print(AVAIBILITY_STATESHEET)

 | Qurry version: 0.9.1.dev1
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ......... Python Cython Rust  
 - randomized_measure
   - entangled_core ............... True   True   True  
   - purity_cell .................. True   True   True  
   - wavefunction_overlap ......... True   True   True  
   - echo_cell .................... True   True   True  
 - utils
   - randomized ................... True   True   True  
   - construct .................... True   None   True  
   - dummy ........................ True   None   True  
 - hadamard_test
   - purity_echo_core ............. True   None   True  
 - magnet_square
   - magnsq_core .................. True   None   None  
--------------------------------------------------------
   + True ..... Working normally.
   + False .... Exception occurred.
   + None ..... Not supported.
--------------------------------------------------------



In [23]:
from qurry.tools.qiskit_version import QISKIT_VERSION_STATESHEET

print(QISKIT_VERSION_STATESHEET)

 | Qurry version: 0.9.1.dev1
--------------------------------------------
 ### Qiskit version
 - main
   - qiskit-aer .............. 0.15.0
   - qiskit-aer-gpu .......... 0.14.2
   - qiskit-ibm-provider ..... 0.11.0
   - qiskit-ibm-runtime ...... 0.30.0
 - deprecated
 - into-community
--------------------------------------------
 + qiskit-aer-gpu should have the same version as qiskit-aer.
--------------------------------------------

